In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib.font_manager as fm

# デフォルトフォント設定
font_path = "/System/Library/Fonts/ヒラギノ角ゴシック W6.ttc"
font_prop = fm.FontProperties(fname=font_path)
rcParams['font.family'] = font_prop.get_name()

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import xgboost as xgb
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns


# 7日間ごとにデータを取得し、結合する関数
def get_last_days_data_in_chunks(ticker, interval="1m"):
    end_date = datetime.now()  # 現在の日付を終了日に設定
    start_date = end_date - timedelta(days=1)  # XX日前を開始日に設定
    all_data = pd.DataFrame()  # 取得した全データを格納するデータフレーム

    while start_date < end_date:
        # 7日間の終了日を設定
        period_end = min(start_date + timedelta(days=7), end_date)
        print(f"Fetching data from {start_date.strftime('%Y-%m-%d')} to {period_end.strftime('%Y-%m-%d')}")
        
        # データの取得
        data = yf.download(ticker, interval=interval, start=start_date.strftime('%Y-%m-%d'), end=period_end.strftime('%Y-%m-%d'))
        
        # データを結合
        all_data = pd.concat([all_data, data])
        
        # 次の開始日に移動
        start_date = period_end

    return all_data

# データの取得
ticker = "^N225"
data = get_last_days_data_in_chunks(ticker)

# 取得したデータの確認
print(data)

In [ ]:
def plot_stock_data(data):
    if data.empty:
        print("No data available to plot.")
        return
    
    # 日時をリセット（必要に応じて調整）
    if 'Datetime' in data.columns:
        data['Datetime'] = pd.to_datetime(data['Datetime'])
    else:
        data = data.reset_index()
        data['Datetime'] = pd.to_datetime(data['index']) if 'index' in data.columns else data.index

    # 終値（Close）の可視化
    plt.figure(figsize=(10, 8))
    sns.lineplot(x=data['Datetime'], y=data['Close'], label="終値")
    plt.title("株価データ", fontsize=28)
    plt.xlabel("1分足データ", fontsize=28)
    plt.ylabel("終値", fontsize=28)
    plt.rc('font', size=12)  # デフォルトフォントサイズ
    plt.rc('axes', titlesize=28, labelsize=28)  # タイトルとラベルサイズ
    plt.rc('xtick', labelsize=20)  # X軸目盛りの文字サイズ
    plt.rc('ytick', labelsize=20)  # Y軸目盛りの文字サイズ
    plt.legend(fontsize=24)
    plt.grid(True)
    plt.show()

# 可視化の実行
plot_stock_data(data)


In [ ]:
# 欠損値の確認
missing_values = data.isnull().sum()

print("欠損値の数:\n", missing_values)

# 欠損値のある行を削除
data = data.dropna()

In [ ]:
# 四本値の範囲を確認（負の値や極端に高い値を検出）
for column in ['Open', 'High', 'Low', 'Close', 'Volume']:
    print(f'{column} の異常値:', data[(data[column] < 0) | (data[column] > data[column].mean() + 3 * data[column].std())])

# 異常値を補正（例: 負の値を0に置き換える）
for column in ['Open', 'High', 'Low', 'Close', 'Volume']:
    data[column] = np.where(data[column] < 0, 0, data[column])

In [ ]:
# 必要な列のみを選択して標準化
scaler = StandardScaler()
data[['Open', 'High', 'Low', 'Close', 'Volume']] = scaler.fit_transform(data[['Open', 'High', 'Low', 'Close', 'Volume']])

In [ ]:
# 必要な列のみを選択
features = data[['Open', 'High', 'Low', 'Close', 'Volume']]

# 1. 5期間移動平均
data['SMA_5'] = data['Close'].rolling(window=5).mean()

# 2. 20期間移動平均
data['SMA_20'] = data['Close'].rolling(window=20).mean()

# 4. 短期・中期移動平均差分（5期間 - 20期間）
data['SMA_diff_5_20'] = data['SMA_5'] - data['SMA_20']

# 5. ボリンジャーバンド上限（20期間）
data['BB_upper'] = data['SMA_20'] + (data['Close'].rolling(window=20).std() * 2)

# 6. ボリンジャーバンド下限（20期間）
data['BB_lower'] = data['SMA_20'] - (data['Close'].rolling(window=20).std() * 2)

# 7. ボリンジャーバンド幅
data['BB_width'] = data['BB_upper'] - data['BB_lower']

# 8. 標準偏差（20期間）
data['Volatility_20'] = data['Close'].rolling(window=20).std()

# 9. ATR（平均値レンジ）
data['Prev_Close'] = data['Close'].shift(1)  # 前日の終値を新しい列に保存

data['TR'] = data[['High', 'Low', 'Prev_Close']].apply(
    lambda x: max(
        x['High'] - x['Low'],
        abs(x['High'] - x['Prev_Close']),
        abs(x['Low'] - x['Prev_Close'])
    ),
    axis=1
)

data['ATR_14'] = data['TR'].rolling(window=14).mean()

# 'Prev_Close'列は不要なので削除する場合は次の行を追加
data.drop(columns=['Prev_Close'], inplace=True)


# 10. RSI（14期間）
delta = data['Close'].diff(1)
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
data['RSI_14'] = 100 - (100 / (1 + rs))

# 11. MACD
data['EMA_12'] = data['Close'].ewm(span=12, adjust=False).mean()
data['EMA_26'] = data['Close'].ewm(span=26, adjust=False).mean()
data['MACD'] = data['EMA_12'] - data['EMA_26']

# 12. MACDシグナル
data['MACD_signal'] = data['MACD'].ewm(span=9, adjust=False).mean()

# 13. MACDヒストグラム
data['MACD_histogram'] = data['MACD'] - data['MACD_signal']

# 14. ストキャスティクス（%K）
data['L14'] = data['Low'].rolling(window=14).min()
data['H14'] = data['High'].rolling(window=14).max()
data['%K'] = (data['Close'] - data['L14']) * 100 / (data['H14'] - data['L14'])

# 15. ストキャスティクス（%D）
data['%D'] = data['%K'].rolling(window=3).mean()

# 17. 移動平均の傾き（20期間）
data['SMA_slope_20'] = data['SMA_20'].diff()

# 18. サポートラインとの距離
data['Support_Distance'] = data['Low'] - data['Low'].rolling(window=20).min()

# 19. レジスタンスラインとの距離
data['Resistance_Distance'] = data['High'].rolling(window=20).max() - data['High']

# 20. 価格変化率（5期間）
data['Price_Change_5'] = data['Close'].pct_change(periods=5)

# 21. 価格変化率（20期間）
data['Price_Change_20'] = data['Close'].pct_change(periods=20)

# 22. 瞬間ボラティリティ（差分平方）
data['Instant_Volatility'] = data['Close'].diff().pow(2).rolling(window=5).sum()

# 23. エンベロープ（±2%）
data['Envelope_upper'] = data['SMA_20'] * 1.02
data['Envelope_lower'] = data['SMA_20'] * 0.98

# 24. DMIのADX（14期間）
data['Plus_DM'] = data['High'].diff()
data['Minus_DM'] = -data['Low'].diff()
data['Plus_DI'] = 100 * (data['Plus_DM'].where(data['Plus_DM'] > 0, 0)).ewm(alpha=1/14, min_periods=0).mean() / data['ATR_14']
data['Minus_DI'] = 100 * (data['Minus_DM'].where(data['Minus_DM'] > 0, 0)).ewm(alpha=1/14, min_periods=0).mean() / data['ATR_14']
data['DX'] = (abs(data['Plus_DI'] - data['Minus_DI']) / abs(data['Plus_DI'] + data['Minus_DI'])) * 100
data['ADX'] = data['DX'].ewm(alpha=1/14, min_periods=0).mean()

# 25. 移動平均乖離率（20期間）
data['MA_deviation_20'] = (data['Close'] - data['SMA_20']) / data['SMA_20']

# 26. ウィリアムズ％R（14期間）
data['Williams_R'] = (data['H14'] - data['Close']) / (data['H14'] - data['L14']) * -100

# 27. 終値/始値の比率
data['Close_to_Open'] = data['Close'] / data['Open']

# 28. 終値の変化幅（前日比）
data['Close_Change'] = data['Close'].diff()

# 29. 終値の変化率（前日比、パーセンテージ）
data['Close_Pct_Change'] = data['Close'].pct_change() * 100

# 30. 前のローソク足との比率
data['Previous_Ratio'] = data['Close'] / data['Close'].shift(1)

# 欠損値の発生する先頭行を削除
data = data.dropna()

print(data)

In [ ]:
# 終値の1日後の値を追加
data['Next_Close'] = data['Close'].shift(-1)

# 終値が上がるか下がるかのラベルを作成
#　Tureなら1、Falseなら0
data.loc[:, 'Price_Up'] = (data['Next_Close'] > data['Close']).astype(int)

# 最後の行は次の値がないため削除
data = data.dropna()

print(data[[ 'Close', 'Next_Close', 'Price_Up']].head())

In [ ]:
columns_to_scale = ['Open', 'High', 'Low', 'Close', 'Volume', 'SMA_5', 'SMA_20', 'SMA_diff_5_20', 
                    'BB_upper', 'BB_lower', 'BB_width', 'Volatility_20', 'ATR_14', 'RSI_14', 
                    'MACD', 'MACD_signal', 'MACD_histogram', '%K', '%D', 'SMA_slope_20', 
                    'Support_Distance', 'Resistance_Distance', 'Price_Change_5', 'Price_Change_20', 
                    'Instant_Volatility', 'Envelope_upper', 'Envelope_lower', 'Plus_DI', 
                    'Minus_DI', 'DX', 'ADX', 'MA_deviation_20', 'Williams_R', 'Close_to_Open', 
                    'Close_Change', 'Close_Pct_Change', 'Previous_Ratio']

# 特徴量とラベルの定義
X = data[columns_to_scale]
y = data['Price_Up']

In [ ]:
#単純なランダムフォレストによる勝率の計算
# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ランダムフォレストの学習と予測
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

# 正解率の計算
accuracy = accuracy_score(y_test, y_pred)
print(f'ランダムフォレストのAccuracy: {accuracy:.3f}')

In [ ]:
from sklearn.tree import export_graphviz
import graphviz

# 決定木の可視化（1本の決定木を抽出）
estimator = rf_model.estimators_[0]

# 決定木の可視化用データを作成
dot_data = export_graphviz(estimator, out_file=None, 
                           feature_names=columns_to_scale, 
                           class_names=['Down', 'Up'], 
                           filled=True, rounded=True, 
                           special_characters=True)

# Graphvizで描画
graph = graphviz.Source(dot_data)
graph.render("decision_tree")  # PDFとして出力
graph


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate_precision_recall_f1(y_true, y_pred):
    precision = precision_score(y_true, y_pred, average='binary')
    recall = recall_score(y_true, y_pred, average='binary')
    f1 = f1_score(y_true, y_pred, average='binary')
    
    print(f"適合率 (Precision): {precision:.3f}")
    print(f"再現率 (Recall): {recall:.3f}")
    print(f"F1スコア: {f1:.3f}")
    
# 基礎モデルの評価
evaluate_precision_recall_f1(y_test, y_pred)

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

# ==============================
# Step 1: 特徴量選定 (RFE)
# ==============================
# ランダムフォレストモデルを初期化
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# RFEで特徴量を選定（10個）
rfe = RFE(estimator=rf_model, n_features_to_select=10)
rfe.fit(X, y)

# 選ばれた特徴量
selected_features_rfe = X.columns[rfe.support_]
print("RFEで選ばれた特徴量:", selected_features_rfe)

# 選ばれた特徴量だけでデータを再構築
X_rfe_selected = X[selected_features_rfe]

# ==============================
# Step 2: ハイパーパラメータチューニング
# ==============================
# データを訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X_rfe_selected, y, test_size=0.2, random_state=42, shuffle=False)

# ハイパーパラメータの設定
param_grid = {
    'n_estimators': [50, 100, 200],      # 木の本数
    'max_depth': [None, 10, 20],         # 木の深さ
    'min_samples_split': [2, 5, 10],     # 分割するための最小サンプル数
    'min_samples_leaf': [1, 2, 4],       # 葉ノードの最小サンプル数
    'max_features': ['sqrt', 'log2']     # 各分割で考慮する特徴量の数
}

# GridSearchCVを初期化
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, 
                    cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

# ハイパーパラメータチューニングの実行
grid_search.fit(X_train, y_train)

# 最適なハイパーパラメータの表示
best_params = grid_search.best_params_
print(f'最適なハイパーパラメータ: {best_params}')

# ==============================
# Step 3: 最適パラメータでモデルを再構築
# ==============================
# 最適なパラメータを使ってモデルを再構築
best_rf_model = RandomForestClassifier(**best_params, random_state=42)
best_rf_model.fit(X_train, y_train)

# テストデータで予測
y_pred = best_rf_model.predict(X_test)

# 勝率（Accuracy）を計算
accuracy = accuracy_score(y_test, y_pred)
print(f'ランダムフォレストモデルの勝率（Accuracy）: {accuracy:.3f}')

# ==============================
# Step 4: 勝率の詳細な確認（上がる・下がる）
# ==============================
import numpy as np

# 上がる・下がるの予測勝率をそれぞれ計算
actual_up = (y_test == 1)  # 実際に値が上がった場合
actual_down = (y_test == 0)  # 実際に値が下がった場合

pred_up_correct = np.sum((y_pred == 1) & actual_up)
pred_down_correct = np.sum((y_pred == 0) & actual_down)

up_accuracy = pred_up_correct / np.sum(actual_up) if np.sum(actual_up) > 0 else 0
down_accuracy = pred_down_correct / np.sum(actual_down) if np.sum(actual_down) > 0 else 0

print(f'値上がりを当てた勝率: {up_accuracy:.3f}')
print(f'値下がりを当てた勝率: {down_accuracy:.3f}')


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate_precision_recall_f1(y_true, y_pred):
    precision = precision_score(y_true, y_pred, average='binary')
    recall = recall_score(y_true, y_pred, average='binary')
    f1 = f1_score(y_true, y_pred, average='binary')
    
    print(f"適合率 (Precision): {precision:.3f}")
    print(f"再現率 (Recall): {recall:.3f}")
    print(f"F1スコア: {f1:.3f}")
    
# 特徴量選定 + ハイパーパラメータの設定後の評価
evaluate_precision_recall_f1(y_test, y_pred)


In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
# 特徴量選定 (RFE)
rfe = RFE(estimator=RandomForestClassifier(n_estimators=100, random_state=42), n_features_to_select=10)
rfe.fit(X, y)
selected_features_rfe = X.columns[rfe.support_]
X_rfe_selected = X[selected_features_rfe]

# データ分割
X_train, X_test, y_train, y_test = train_test_split(X_rfe_selected, y, test_size=0.2, random_state=42, shuffle=False)

# デフォルト設定のランダムフォレスト
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

# 勝率計算
accuracy = accuracy_score(y_test, y_pred)
print(f'特徴量選定のみ実施時の勝率: {accuracy:.3f}')


In [ ]:
# 特徴量選定のみでの評価
evaluate_precision_recall_f1(y_test, y_pred)

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
# データ分割（元の全特徴量を使用）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# ハイパーパラメータの設定とGridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42), 
    param_grid=param_grid, 
    cv=5, 
    scoring='accuracy', 
    n_jobs=-1, 
    verbose=2
)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_

# 最適パラメータでモデルを構築
best_rf_model = RandomForestClassifier(**best_params, random_state=42)
best_rf_model.fit(X_train, y_train)
y_pred = best_rf_model.predict(X_test)

# 勝率計算
accuracy = accuracy_score(y_test, y_pred)
print(f'ハイパーパラメータ設定のみ実施時の勝率: {accuracy:.3f}')


In [ ]:
# ハイパーパラメータ設定のみでの評価
evaluate_precision_recall_f1(y_test, y_pred)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# データ分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# ランダムフォレストの学習
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# 特徴量重要度を取得
feature_importances = rf_model.feature_importances_
feature_names = X.columns

# 特徴量重要度をデータフレームにまとめる
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# 結果を表示
print("特徴量の重要度:\n", importance_df)

# 重要度の高い特徴量を選ぶ（しきい値を設定）
threshold = 0.01  # 重要度が0.01以上のものを選択
selected_features = importance_df[importance_df['Importance'] > threshold]['Feature'].tolist()

print("\n選ばれた特徴量:", selected_features)

# 選ばれた特徴量でデータセットを作成
X_selected = X[selected_features]


In [ ]:
#クロスバリエーションを組み込んだランダムフォレストによる勝率の計算

# 時系列クロスバリデーションの設定
tscv = TimeSeriesSplit(n_splits=5)

# モデルの初期化
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# クロスバリデーションの実行
cv_scores = []
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # モデルの訓練
    rf_model.fit(X_train, y_train)
    
    # テストデータで予測
    y_pred = rf_model.predict(X_test)
    
    # 正解率の計算
    accuracy = accuracy_score(y_test, y_pred)
    cv_scores.append(accuracy)

# 各分割のスコアと平均スコアを表示
print(f'各FoldのAccuracy: {cv_scores}')
print(f'時系列クロスバリデーションの平均Accuracy: {np.mean(cv_scores):.3f}')

In [ ]:
#クロスバリエーション×ハイパラメータの設定×ランダムフォレスト
#めっちゃ時間かかるからあんまりやりたくない。

# 時系列クロスバリデーションの設定
tscv = TimeSeriesSplit(n_splits=5)

# ハイパーパラメータの設定
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

# ランダムフォレストの初期化
rf_model = RandomForestClassifier(random_state=42)

# GridSearchCVの初期化
grid_search = GridSearchCV(
    estimator=rf_model, 
    param_grid=param_grid, 
    cv=tscv, 
    scoring='accuracy', 
    n_jobs=-1, 
    verbose=2
)

# ハイパーパラメータチューニングの実行
grid_search.fit(X, y)

# 最適なモデルとそのスコアを表示
print(f'最適なハイパーパラメータ: {grid_search.best_params_}')
print(f'最適なモデルの平均Accuracy: {grid_search.best_score_:.3f}')


In [ ]:
#チューニング後に値を入れ直すためのコード
best_params = {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 50}


rf_model_optimized = RandomForestClassifier(**best_params, random_state=42)
rf_model_optimized.fit(X_train, y_train)

y_pred = rf_model_optimized.predict(X_test)
final_accuracy = accuracy_score(y_test, y_pred)
print(f'最適ハイパーパラメータモデルのAccuracy: {final_accuracy:.3f}')


In [ ]:
#xgboostで、時短してみた。1つ上と内容は同様のはず。

from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
import numpy as np

# データの欠損値を処理
X = X.fillna(0)  # 特徴量の欠損値を処理
y = y.dropna()   # ラベルの欠損値を削除

# ラベルの不均衡を修正
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# 時系列クロスバリデーションの設定
tscv = TimeSeriesSplit(n_splits=3)

# XGBoostモデルの初期化
xgb_model = XGBClassifier(random_state=42, tree_method='hist')

# パラメータグリッドの設定
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# GridSearchCVの初期化
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    cv=tscv,
    scoring='accuracy',
    n_jobs=-1,
    verbose=3,
    error_score='raise'
)

# ハイパーパラメータチューニングの実行
grid_search.fit(X_resampled, y_resampled)

# 最適なモデルとそのスコアを表示
print(f'最適なハイパーパラメータ: {grid_search.best_params_}')
print(f'最適なモデルの平均Accuracy: {grid_search.best_score_:.3f}')

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

# データを訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# 最適なハイパーパラメータでモデルを再構築
best_params = {'colsample_bytree': 1.0, 'learning_rate': 0.2, 'max_depth': 10, 'n_estimators': 200, 'subsample': 0.8}
final_model = XGBClassifier(random_state=42, tree_method='hist', **best_params)

# モデルの訓練
final_model.fit(X_train, y_train)

# テストデータで予測
y_pred = final_model.predict(X_test)

# 勝率（Accuracy）を計算
accuracy = accuracy_score(y_test, y_pred)
print(f'最終モデルのテストデータでの勝率（Accuracy）: {accuracy:.3f}')

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

# 実際の株価が上がったか下がったかを表すデータ
# 上がった場合: 1, 下がった場合: 0
actual = y_test.to_numpy()  # 実際の値 (0: 下がる, 1: 上がる)

# 予測結果
predictions = y_pred  # モデルの予測 (0: 下がる, 1: 上がる)

# 値上がり予測での勝率（ロングポジション）
long_indices = np.where(predictions == 1)[0]  # 上がると予測したインデックス
long_successes = np.sum(actual[long_indices] == 1)  # 実際に上がった数
long_accuracy = long_successes / len(long_indices) if len(long_indices) > 0 else 0.0

# 値下がり予測での勝率（ショートポジション）
short_indices = np.where(predictions == 0)[0]  # 下がると予測したインデックス
short_successes = np.sum(actual[short_indices] == 0)  # 実際に下がった数
short_accuracy = short_successes / len(short_indices) if len(short_indices) > 0 else 0.0

# 全体の勝率
overall_accuracy = accuracy_score(actual, predictions)

# 結果を表示
print(f"ロングポジションでの勝率: {long_accuracy:.3f}")
print(f"ショートポジションでの勝率: {short_accuracy:.3f}")
print(f"全体の勝率: {overall_accuracy:.3f}")
